## Import packages

In [ ]:
import ck.kernel as ck
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import json
import os

In [ ]:
print "Collective Knowledge: v%s" % ck.version({})['version_str']
print "pandas: v%s" % pd.version.version
print "NumPy: v%s" % np.version.version
print "Matplotlib: v%s" % matplotlib.__version__
print "JSON: v%s" % json.__version__

In [ ]:
%matplotlib inline

## Find results

In [ ]:
dataset = 'SGEMM_NT'
data_uoa =  dataset + '-explore-f-n'
module_uoa = 'experiment'

In [ ]:
r=ck.access({'action':'list_points', 'module_uoa':module_uoa, 'data_uoa':data_uoa})
if r['return']>0:
  print ("Error: %s" % r['error'])
  exit(1)

## Show results

In [ ]:
data_list  = []
index_list = []

for point in r['points']:
    with open(os.path.join(r['path'], 'ckp-%s.flat.json' % point)) as point_file:
        point_data = json.load(point_file)
    # Data.    
    diff = point_data.get("##characteristics#run#run_time_state#RESULTS#max_abs_diff#all")
    match = point_data.get("##characteristics#run#run_time_state#RESULTS#match#all")
    diff_match = [x for t in zip(diff, match) for x in t]
    data_list.append(diff_match)
    
    # Row index.
    cl_file = point_data.get("##characteristics#run#run_time_state#METADATA#file#all_unique")[0]
    matrix_order = point_data.get("##characteristics#run#run_time_state#CMD_LINE_ARGS#matrix_order#all_unique")[0]
    index_list.append((cl_file, matrix_order))

In [ ]:
num_repetitions = 4 # TODO: get from points
cols_i = ['Max abs diff', 'Match?'] * num_repetitions
rows_mi = pd.MultiIndex.from_tuples(names=['OpenCL program', 'Matrix order'], tuples=index_list)
df = pd.DataFrame(data=data_list, index=rows_mi, columns=cols_i).sortlevel()
df

#### Dump to files

In [ ]:
with open('%s-match_tmp.tex' % data_uoa, 'w') as tex_file:
    tex_file.write(df.to_latex())